In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'co2': 0.0004, 'o3': 'atmpro'}
band = [7]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.55, 0.9)]
o co2 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]
o o3 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.028424                    1 -0.016485
0.000750        2 -0.029896                    2 -0.016922
0.001052        3 -0.029747                    3 -0.015781
0.001476        4 -0.024956                    4 -0.011247
0.002070        5 -0.014955                    5 -0.002903
0.002904        6  0.000953                    6  0.009764
0.004074        7  0.023542                    7  0.027324
0.005714        8  0.053633                    8  0.050520
0.008015        9  0.092044                    9  0.079976
0.011243       10  0.139471                   10  0.116426
0.015771       11  0.197004                   11  0.160857
0.022122       12  0.265319                   12  0.213979
0.031031       13  0.344921                   13  0.276463
0.043528       14  0.464881                   14  0.372595
0.061057       15  0.643067                   15  0.519241
0.085645       16  0.848591                   16  0.695010
0.120136       17  1.066488                   17  0.891282
0.168516       18  1.289810                   18  1.106116
0.236378       19  1.511258                   19  1.335185
0.331549       20  1.727284                   20  1.573582
0.465100       21  1.930001                   21  1.802296
0.652400       22  1.976326                   22  1.905042
0.915100       23  1.664982                   23  1.675645
1.283650       24  1.252754                   24  1.284150
1.800600       25  0.928281                   25  0.937519
2.525700       26  0.649576                   26  0.652283
3.542800       27  0.426390                   27  0.436458
4.969550       28  0.285010                   28  0.299870
6.970850       29  0.210987                   29  0.232019
9.778100       30  0.158112                   30  0.179149
13.715850      31  0.112383                   31  0.128989
19.239350      32  0.065525                   32  0.076461
26.987250      33  0.036419                   33  0.042429
37.855300      34  0.023295                   34  0.024722
53.100050      35 -0.000479                   35 -0.001982
73.887500      36 -0.030746                   36 -0.035908
97.662500      37 -0.056523                   37 -0.067220
121.437500     38 -0.075368                   38 -0.090858
145.212500     39 -0.090190                   39 -0.108549
168.987500     40 -0.102751                   40 -0.121648
192.762500     41 -0.102640                   41 -0.117538
216.537500     42 -0.092823                   42 -0.101709
240.312500     43 -0.082534                   43 -0.086146
264.087500     44 -0.069630                   44 -0.069193
287.862500     45 -0.056022                   45 -0.053213
311.637500     46 -0.041172                   46 -0.037820
335.412500     47 -0.028418                   47 -0.025569
359.187500     48 -0.020629                   48 -0.018246
382.962500     49 -0.015016                   49 -0.013023
406.737500     50 -0.010343                   50 -0.008746
430.512500     51 -0.006478                   51 -0.005279
454.287500     52 -0.003610                   52 -0.002739
478.062500     53 -0.001400                   53 -0.000758
501.837500     54  0.000635                   54  0.001067
525.612500     55  0.002558                   55  0.002780
549.387500     56  0.004388                   56  0.004413
573.162500     57  0.006118                   57  0.005961
596.937500     58  0.007748                   58  0.007432
620.712500     59  0.009282                   59  0.008829
644.487500     60  0.010699                   60  0.010123
668.262500     61  0.011979                   61  0.011287
692.037500     62  0.013149                   62  0.012365
715.812500     63  0.014221                   63  0.013365
739.587500     64  0.015172                   64  0.014267
763.362500     65  0.016012                   65  0.015075
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -10.944362  0.000000 -10.944362 -10.659395 -5.550883e-08   
0.000624    2     -10.944366  0.000002 -10.944364 -10.659398  1.549868e-06   
0.000876    3     -10.944368  0.000003 -10.944365 -10.659399  2.332768e-06   
0.001229    4     -10.944372  0.000006 -10.944366 -10.659402  3.871579e-06   
0.001723    5     -10.944377  0.000010 -10.944368 -10.659405  6.767222e-06   
0.002417    6     -10.944386  0.000017 -10.944369 -10.659411  1.204143e-05   
0.003391    7     -10.944399  0.000030 -10.944369 -10.659419  2.142502e-05   
0.004757    8     -10.944418  0.000053 -10.944365 -10.659431  3.780627e-05   
0.006672    9     -10.944445  0.000092 -10.944353 -10.659448  6.599184e-05   
0.009359    10    -10.944481  0.000158 -10.944324 -10.659470  1.139187e-04   
0.013128    11    -10.944530  0.000269 -10.944261 -10.659499  1.945810e-04   
0.018415    12    -10.944591  0.000453 -10.944138 -10.659533  3.293026e-04   
0.025830    13    -10.944661  0.000756 -10.943905 -10.659568  5.525104e-04   
0.036232    14    -10.944731  0.001251 -10.943480 -10.659594  9.195439e-04   
0.050823    15    -10.944789  0.002113 -10.942676 -10.659596  1.564789e-03   
0.071291    16    -10.944814  0.003697 -10.941117 -10.659537  2.765158e-03   
0.100000    17    -10.944736  0.006505 -10.938231 -10.659339  4.931337e-03   
0.140271    18    -10.944397  0.011253 -10.933144 -10.658839  8.683525e-03   
0.196760    19    -10.943483  0.018970 -10.924513 -10.657727  1.497343e-02   
0.275997    20    -10.941392  0.031064 -10.910328 -10.655424  2.520398e-02   
0.387100    21    -10.936946  0.049351 -10.887595 -10.650843  4.133537e-02   
0.543100    22    -10.927846  0.075916 -10.851930 -10.642019  6.582001e-02   
0.761700    23    -10.912307  0.111554 -10.800754 -10.627354  1.004905e-01   
1.068500    24    -10.891922  0.151678 -10.740244 -10.607983  1.420230e-01   
1.498800    25    -10.868108  0.191719 -10.676388 -10.584983  1.844855e-01   
2.102400    26    -10.841072  0.231057 -10.610016 -10.558433  2.249765e-01   
2.949000    27    -10.813740  0.268868 -10.544872 -10.531279  2.632433e-01   
4.136600    28    -10.791267  0.306379 -10.484888 -10.508554  3.019257e-01   
5.802500    29    -10.777290  0.348645 -10.428645 -10.493841  3.463946e-01   
8.139200    30    -10.771045  0.400801 -10.370244 -10.486412  4.031953e-01   
11.417000   31    -10.773417  0.464565 -10.308852 -10.487302  4.736521e-01   
16.014700   32    -10.792004  0.544358 -10.247645 -10.504340  5.609486e-01   
22.464000   33    -10.845465  0.647878 -10.197587 -10.556399  6.714275e-01   
31.510500   34    -10.952024  0.793465 -10.158559 -10.662640  8.231402e-01   
44.200100   35    -11.118585  0.995043 -10.123543 -10.831912  1.029578e+00   
62.000000   36    -11.364229  1.239677 -10.124552 -11.088632  1.282119e+00   
85.775000   37    -11.708150  1.497008 -10.211142 -11.461847  1.554194e+00   
109.550000  38    -12.052830  1.682501 -10.370330 -11.851300  1.754317e+00   
133.325000  39    -12.390404  1.807813 -10.582591 -12.245390  1.892496e+00   
157.100000  40    -12.732933  1.896337 -10.836596 -12.650756  1.992123e+00   
180.875000  41    -13.090674  1.964699 -11.125975 -13.071363  2.070093e+00   
204.650000  42    -13.430075  2.015033 -11.415042 -13.460459  2.128131e+00   
228.425000  43    -13.727719  2.051259 -11.676460 -13.789092  2.170289e+00   
252.200000  44    -13.987355  2.078454 -11.908901 -14.063610  2.202165e+00   
275.975000  45    -14.203746  2.098746 -12.105000 -14.282395  2.226060e+00   
299.750000  46    -14.377223  2.114446 -12.262777 -14.450603  2.244388e+00   
323.525000  47    -14.508211  2.129480 -12.378731 -14.573511  2.260772e+00   
347.300000  48    -14.605375  2.146611 -12.458765 -14.662724  2.277967e+00   
371.075000  49    -14.683633  2.166771 -12.5168

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')